In [1]:
#import dependencies
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
import pandas as pd
import os
# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float,ForeignKey 

import pymysql
pymysql.install_as_MySQLdb()

In [4]:
data_df=pd.read_csv("Cleaned_Data.csv",low_memory=False)

data_df=data_df.rename(columns={"Price Maximum": "PriceMaximum", "Price Minimum": "PriceMinimum","Start Date":"StartDate"})
#data_df=data_df.drop(['Unnamed: 10'], axis=1)
data_df.head()

,Genre,Latitude,Longitude,Name,PriceMaximum,PriceMinimum,StartDate,Subgenre,Venue,VenueID
0,Rock,41.506036,-81.699570,Taylor Swift reputation Stadium Tour,495.5,45.5,7/17/18,Pop,"""FirstEnergy Stadium, Home of the Cleveland Br...",KovZpZAFdEJA
1,Metal,41.542609,-87.771175,Ozzy Osbourne: No More Tours 2,250.0,29.5,9/21/18,Heavy Metal,"""Hollywood Casino Amphitheatre - Chicago, IL""",KovZpZAEktFA
2,Rock,41.542609,-87.771175,Lynyrd Skynyrd: Last of the Street Survivors F...,199.5,29.5,8/3/18,Pop,"""Hollywood Casino Amphitheatre - Chicago, IL""",KovZpZAEktFA
3,Rock,41.542609,-87.771175,Imagine Dragons: EVOLVE TOUR,146.0,35.0,7/13/18,Pop,"""Hollywood Casino Amphitheatre - Chicago, IL""",KovZpZAEktFA
4,Pop,41.542609,-87.771175,The Adventures of Kesha and Macklemore,137.0,30.5,7/14/18,Pop,"""Hollywood Casino Amphitheatre - Chicago, IL""",KovZpZAEktFA


In [5]:
#declare the base
Base=declarative_base()

#declare classes and attributes
class Events(Base):
    
    __tablename__="Events"
    id=Column(Integer,primary_key=True)
    genre=Column(String(255))
    Latitude = Column(Float)
    Longitude = Column(Float)
    Name=Column(String(255))
    PriceMaximum = Column(Float)
    PriceMinimum = Column(Float)
    
    StartDate=Column(String(255))
    Subgenre = Column(String(255))
    Venue = Column(String(255))
    VenueID = Column(String(255))
    
   

In [6]:
engine = create_engine(os.path.join("sqlite:///EventsDB.db"))
conn = engine.connect()
Base.metadata.create_all(conn)
session = Session(bind=engine)

In [7]:
data_df.to_sql('Events',conn,if_exists = 'append',index = False)
session.commit()

conn.close()

In [3]:
engine = create_engine(os.path.join("sqlite:///EventsDB.db"),echo=False)


conn = engine.connect()

In [4]:
Base = automap_base()
# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)
# Print all of the classes mapped to the Base
Base.classes.keys()

['Events']

In [6]:
events_db = Base.classes.Events

session = Session(engine)

In [7]:
first_row = session.query(events_db).first()
#first_row.__dict__

In [8]:
result_sample = session.query(events_db.Latitude,events_db.Longitude,events_db.Name,events_db.PriceMaximum,\
                       events_db.PriceMinimum,events_db.StartDate,events_db.Subgenre,events_db.Venue).all()

In [7]:
#result_sample[0]

In [9]:
result_list=[]
result_dict={}
names = ["Latitude", "Longitude", "Name","PriceMaximum","PriceMinimum","StartDate","Subgenre","Venue"]
for each in result_sample:
    result_dict={}
    for i in range(len(names)):
        result_dict[names[i]]=each[i]
        #print(each[1])
    result_list.append(result_dict)

In [10]:
#result_list

In [12]:
#result_samples_desc = session.query(samples_db.otu_id,getattr(samples_db, s)).order_by(getattr(samples_db, s).desc()).all()
#result_samples_desc

In [13]:
url="/maps/StartDate/7/7/18"
criteria=url.split("/")
print(criteria[2])
d=criteria[3]+"/"+criteria[4]+"/"+criteria[5]
print(d)
result_wfreq= session.query(events_db.Latitude,events_db.Longitude,events_db.Name,events_db.PriceMaximum,\
                       events_db.PriceMinimum,events_db.StartDate,events_db.Subgenre,events_db.Venue).filter(getattr(events_db,criteria[2])==d).all()
#result_wfreq

StartDate
7/7/18


In [45]:
session = Session(engine)
result = session.execute("select genre,name,max(PriceMaximum) , startdate,venue from Events where PriceMaximum<> 0  group by genre order by max(PriceMaximum) desc").fetchall()

In [46]:
result

[('Pop', 'Yanni 25 - Acropolis Anniversary Concert Tour', 3200.0, '7/1/18', 'Mystic Lake Casino Hotel'),
 ('Metal', 'Metallica - WorldWired Tour', 2499.0, '9/8/18', 'Alerus Center'),
 ('Hip-Hop/Rap', 'JAY-Z and BEYONCÉ - OTR II', 1993.0, '8/8/18', 'U.S. Bank Stadium'),
 ('Rock', 'Bon Jovi - This House Is Not For Sale - Tour', 1666.5, '4/28/18', 'Xcel Energy Center'),
 ('Country', 'Tim McGraw & Faith Hill Soul2Soul The World Tour 2018', 1299.0, '6/1/18', 'Charleston Civic Center'),
 ('Dance/Electronic', 'Global Dance Festival - Two Day Pass', 500.0, '7/20/18', 'Sports Authority Field At Mile High')]

In [49]:
result_list=[]
result_dict={}
names = ["Genre","Name", "PriceMaximum","StartDate","Venue"]
for each in result:
    result_dict={}
    for i in range(len(names)):
        result_dict[names[i]]=each[i]
        #print(each[1])
    result_list.append(result_dict)

In [50]:
result_list

[{'Genre': 'Pop',
  'Name': 'Yanni 25 - Acropolis Anniversary Concert Tour',
  'PriceMaximum': 3200.0,
  'StartDate': '7/1/18',
  'Venue': 'Mystic Lake Casino Hotel'},
 {'Genre': 'Metal',
  'Name': 'Metallica - WorldWired Tour',
  'PriceMaximum': 2499.0,
  'StartDate': '9/8/18',
  'Venue': 'Alerus Center'},
 {'Genre': 'Hip-Hop/Rap',
  'Name': 'JAY-Z and BEYONCÉ - OTR II',
  'PriceMaximum': 1993.0,
  'StartDate': '8/8/18',
  'Venue': 'U.S. Bank Stadium'},
 {'Genre': 'Rock',
  'Name': 'Bon Jovi - This House Is Not For Sale - Tour',
  'PriceMaximum': 1666.5,
  'StartDate': '4/28/18',
  'Venue': 'Xcel Energy Center'},
 {'Genre': 'Country',
  'Name': 'Tim McGraw & Faith Hill Soul2Soul The World Tour 2018',
  'PriceMaximum': 1299.0,
  'StartDate': '6/1/18',
  'Venue': 'Charleston Civic Center'},
 {'Genre': 'Dance/Electronic',
  'Name': 'Global Dance Festival - Two Day Pass',
  'PriceMaximum': 500.0,
  'StartDate': '7/20/18',
  'Venue': 'Sports Authority Field At Mile High'}]

In [29]:
session = Session(engine)
result = session.execute("select venue,name, startdate,genre,min(PriceMinimum) from Events where PriceMinimum<> 0 group by genre order by min(PriceMinimum)").fetchall()

[{'Genre': 45.5,
  'Name': -81.69956958,
  'PriceMaximum': 'Taylor Swift reputation Stadium Tour',
  'StartDate': 495.5,
  'Venue': 41.50603554},
 {'Genre': 29.5,
  'Name': -87.7711753,
  'PriceMaximum': 'Ozzy Osbourne: No More Tours 2',
  'StartDate': 250.0,
  'Venue': 41.542609399999996},
 {'Genre': 29.5,
  'Name': -87.7711753,
  'PriceMaximum': 'Lynyrd Skynyrd: Last of the Street Survivors Farewell Tour',
  'StartDate': 199.5,
  'Venue': 41.542609399999996},
 {'Genre': 35.0,
  'Name': -87.7711753,
  'PriceMaximum': 'Imagine Dragons: EVOLVE TOUR',
  'StartDate': 146.0,
  'Venue': 41.542609399999996},
 {'Genre': 30.5,
  'Name': -87.7711753,
  'PriceMaximum': 'The Adventures of Kesha and Macklemore',
  'StartDate': 137.0,
  'Venue': 41.542609399999996},
 {'Genre': 29.5,
  'Name': -87.7711753,
  'PriceMaximum': 'End Of The World Tour: Avenged Sevenfold With Prophets Of Rage',
  'StartDate': 136.0,
  'Venue': 41.542609399999996},
 {'Genre': 18.75,
  'Name': -87.7711753,
  'PriceMaximum':

In [30]:
result

[('The Rose', 'Accept', '9/29/18', 'Metal', 1.0),
 ('Baxter Arena', 'Logic', '4/4/18', 'Hip-Hop/Rap', 5.0),
 ('House of Blues San Diego', 'SDSU vs UCSD DJ Battle', '4/6/18', 'Dance/Electronic', 7.0),
 ('Starplex Pavilion', '97.1 The Eagle Presents BFD 2018', '5/26/18', 'Rock', 9.71),
 ('8 Seconds Saloon', 'Frankie Ballard', '4/20/18', 'Country', 10.0),
 ('Worthington United Methodist Church', 'Illuminati Columbus Gay Mens Chorus', '5/6/18', 'Pop', 10.0)]

In [55]:
session = Session(engine)
result = session.execute("SELECT distinct genre, name,Max(PriceMaximum - PriceMinimum) AS Gap FROM Events  group by name order by Gap desc").fetchall()

In [56]:
result

[('Pop', 'Yanni 25 - Acropolis Anniversary Concert Tour', 3159.5),
 ('Metal', 'Metallica - WorldWired Tour', 2437.0),
 ('Hip-Hop/Rap', 'JAY-Z and BEYONCÉ - OTR II', 1973.0),
 ('Rock', 'Bon Jovi - This House Is Not For Sale - Tour', 1640.0),
 ('Rock', 'Taylor Swift reputation Stadium Tour', 1450.5),
 ('Rock', 'Elton John: Farewell Yellow Brick Road', 1415.5),
 ('Country', 'Tim McGraw & Faith Hill Soul2Soul The World Tour 2018', 1252.25),
 ('Country', 'The 2018 Country Megaticket Presented By Pennzoil', 1076.0),
 ('Rock', 'Justin Timberlake - The Man Of The Woods Tour', 1026.5),
 ('Country', '2018 Jiffy Lube Live Country Megaticket presented by Pennzoil', 1000.0),
 ('Metal', 'Fozzy & Iced Earth - Special Ticket Combo Offer', 970.5),
 ('Rock', 'Lynyrd Skynyrd: Last of the Street Survivors Farewell Tour', 952.75),
 ('Country', 'Country Megaticket 2018', 950.0),
 ('Country', '2018 Country Megaticket', 940.0),
 ('Country', 'George Strait: Strait Down Route 66', 920.25),
 ('Country', '2018 Sh

In [57]:
result = session.execute("SELECT distinct genre, name,Max(PriceMaximum - PriceMinimum),startdate,venue AS Gap FROM Events  group by name order by Gap desc").fetchall()

In [58]:
result

[('Rock', 'Backstreet Boys: Larger Than Life', 160.0, '10/27/18', 'Zappos Theater at Planet Hollywood'),
 ('Country', '2018 Xfinity Center Country Megaticket', 550.0, '12/1/18', 'Xfinity Center'),
 ('Hip-Hop/Rap', 'Post Malone', 79.0, '5/24/18', 'Xfinity Center'),
 ('Rock', 'Bon Jovi - This House Is Not For Sale - Tour', 1640.0, '4/28/18', 'Xcel Energy Center'),
 ('Rock', 'Bruno Mars: 24K Magic World Tour', 740.5, '9/11/18', 'Xcel Energy Center'),
 ('Rock', 'Daryl Hall & John Oates and Train', 648.75, '5/16/18', 'Xcel Energy Center'),
 ('Pop', 'Harry Styles Live On Tour', 495.5, '7/1/18', 'Xcel Energy Center'),
 ('Pop', 'Josh Groban: Bridges Tour w/ very special guest Idina Menzel', 626.5, '11/2/18', 'Xcel Energy Center'),
 ('Rock', 'Maroon 5: Red Pill Blues Tour 2018', 645.5, '9/18/18', 'Xcel Energy Center'),
 ('Rock', 'Paul Simon: Homeward Bound The Farewell Performance', 485.5, '6/8/18', 'Xcel Energy Center'),
 ('Country', 'Shania Twain: NOW', 570.05, '5/15/18', 'Xcel Energy Center'